# Interpreter Activity 1

Find the bugs in this code!

In [1]:
data Val = IntVal Integer
         | BoolVal Bool
   deriving (Show,Eq)

In [2]:
data Exp = IntExp Integer
         | IntOpExp String Exp Exp
         | CompOpExp String Exp Exp
         | IfExp Exp Exp Exp
         | LetExp String Exp Exp -- "let x = 5 in x" --> LetExp "x" (IntExp 5) (VarExp "x")
         | VarExp String
   deriving (Show,Eq)

In [3]:
type Env = [(String,Val)]

In [4]:
intOps = [ ("+",(+))
         , ("-",(-))
         , ("*",(*))
         , ("/",div)]

In [5]:
compOps = [ (">",(>))
          , ("=",(==))]

In [6]:
liftIntOp f (IntVal i1) (IntVal i2) = IntVal (f i1 i2)
liftIntOp f _           _           = IntVal 0

:t liftIntOp

liftIntOp :: (Integer -> Integer -> Integer) -> Val -> Val -> Val

In [7]:
liftCompOp f (IntVal i1) (IntVal i2) = BoolVal (f i1 i2)
liftCompOp f _           _           = BoolVal False

:t liftCompOp

liftCompOp :: (Integer -> Integer -> Bool) -> Val -> Val -> Val

In [8]:
trumpError s = error $ s ++ " Sad!"

In [9]:
eval :: Exp -> Env -> Val
eval (IntExp i)          _   = IntVal i
eval (VarExp v) env =
   case lookup v env of
     Just val -> val
     Nothing  -> trumpError "You idiot!"
eval (IntOpExp op e1 e2) env =
   let v1 = eval e1 env
       v2 = eval e2 env
       Just f = lookup op intOps
    in liftIntOp f v1 v2
eval (CompOpExp op e1 e2) env =
   let v1 = eval e1 env
       v2 = eval e2 env
       Just f = lookup op compOps
    in liftCompOp f v1 v2
eval (LetExp v e1 e2) env =
   let v1 = eval e1 env
    in eval e2 ((v,v1):env)
eval (IfExp e1 e2 e3) env =
   case eval e1 env of
     BoolVal True -> eval e2 env
     BoolVal False -> eval e3 env
     _   -> trumpError "Nonboolean passed to if!"
     

In [10]:
eval (IntExp 10) []

IntVal 10

In [11]:
eval (IntOpExp "+" (IntOpExp "*" (IntExp 10) (IntExp 2)) (IntExp 22)) []

IntVal 42

In [12]:
:t (/)

(/) :: forall a. Fractional a => a -> a -> a

In [13]:
env1 = [ ("x",IntVal 5), ("y",IntVal 20)]
p1 = IntOpExp "+" (VarExp "x") (VarExp "y")
p2 = IntOpExp "+" (VarExp "x") (VarExp "z")
p3 = IntOpExp "+" (LetExp "z" (IntOpExp "*" (VarExp "x") (VarExp "y")) (VarExp "z")) (VarExp "z") 

In [14]:
eval p3 env1

You idiot! Sad!
CallStack (from HasCallStack):
  error, called at <interactive>:1:16 in interactive:Ghci94

In [15]:
eval p2 env1

You idiot! Sad!
CallStack (from HasCallStack):
  error, called at <interactive>:1:16 in interactive:Ghci94